In [1]:
import json
import pandas as pd
import numpy as np
import os
import string
import eval
from mkdir import mk_dir
from bert_format import bset, train_yn, test_yn

# 训练集

In [2]:
label_path = "/home/data/t070224/Pubmed/Pubmed_Label/3.Orgnize/output/文献合并"
col_dic = {"是否是致病菌":"pat","免疫状态":"immu","预后":"oc"}

for col_name, col_ab in col_dic.items():
    train_qa = []
    # 读取标注信息
    df = pd.read_excel("%s/human_label_pat.xlsx" % (label_path), header=0)
    
    # 设置问题
    if col_name == "是否是致病菌":
        que = []
        for idx in df.index:
            que.append('Can %s cause pneumonia?' % df.loc[idx, "物种名称"])
    if col_name == "免疫状态":
        df = df.loc[df["是否是致病菌"] == "yes", :]
        df.loc[df[col_name] == "抑制", col_name] = "yes"
        que = 'Is patient immunocompromised?'
    if col_name == "预后":
        df = df.loc[df["是否是致病菌"] == "yes", :]
        df.loc[df[col_name] == "死亡", col_name] = "yes"
        que = 'Is patient dead?'
    
    df.loc[df[col_name] != "yes", col_name] = "no"    
    df["question"] = que
    
    # 创建QA样本  
    for idx in df.index:
        train_qa.append(train_yn(
            '%s:%s' % (df.loc[idx, "ID"], df.loc[idx, "物种名称"]),
            df.loc[idx, "question"],
            '%s' % df.loc[idx, col_name],
            '%s' % df.loc[idx, "Full"]))
    bset(train_qa, "input/input_%s/"%(col_ab), "train_set.json")

#  测试集

In [14]:
with open("/home/data/t070224/Pubmed/Pubmed_Search/search_results/search_results_hanimals.txt","r") as f:
    content = f.read()
    spec_papers_dic = json.loads(content)

In [15]:
# 预测样本格式转换
papers_l = {}
for spec_type in spec_papers_dic:

    for spec,spec_papers in spec_papers_dic[spec_type].items():
        
        for paper_id, paper in spec_papers.items():
            
            papers_l['%s:%s' % (paper_id, spec)] = paper


In [24]:
test_qa = []
for paper_id,paper in papers_l.items():     
    if len(paper["texts"]["Ab"]) > 0:
        test_qa.append(
            test_yn(
                paper_id, 
                'Can %s cause Pneumonia?' % paper_id.split(":")[-1],
                paper["texts"]["Ab"]+" "+" ".join(sum(paper["Meshhead"],[]))))
bset(test_qa,"input/input_pat/","test_qa.txt")

# 测试

In [1]:
train_data = pd.read_csv("output/eval_prediction.csv", index_col=0)
train_data = train_data[~train_data.index.duplicated(keep='first')]
standard_0 = pd.read_csv("output/eval_standards.csv", index_col=0)
standard_0 = standard_0[~standard_0.index.duplicated(keep='first')]

In [2]:
pneumonia = {"bronchitis","bronchopneumonia","pneumonia","pneumonitis","pneumonic","pleuropneumonia","pleural"}
LungRelated = {"bronchus","bronchial","tracheobronchial","bronchiolar","bronchopulmonary",
               "bronchoalveolar","bronchoscopy",
               "lung","pulmonary","pneumonic","bronchopulmonary","respiratory","respiration"}
infection = {"infection","infections","abscess","pathology","etiology","diseases",
             "bacteremia","sepsis","septic","pathogenicity","legionellosis","empyema","necrosis"}

In [ ]:
train_extend = {}
for i in train_data.index:
    if i.lower() in papers_l:
        train_extend[i] = papers_l[i.lower()]
    else:
        print(train_data.loc[i])

In [4]:
pats = {}
for paper_id, paper in train_extend.items():
    spec = paper_id.split(":")[1].lower()
    text = paper["texts"]["Ab"] +" "+" ".join(sum(paper["Meshhead"],[])) # 添加mesh词汇
    text = text.translate(str.maketrans(string.punctuation, " "*32,)).replace("  ", " ")
    text = text.lower()
    words = set(text.split(" "))
    if len(words & pneumonia) > 0:
        if "colonization" not in words:
            if "%s / isolation & purification" % spec not in " / ".join(sum(paper["Meshhead"],[])).lower():
                pats[paper_id] = paper
    else:
        if len(words & LungRelated) > 0:
            if len(words & infection) > 0:
                if "colonization" not in words:
                    if "%s / isolation & purification" % spec not in " / ".join(sum(paper["Meshhead"],[])).lower():
                        pats[paper_id] = paper


In [ ]:
cut = 0.5
train_data.loc[train_data["rate"]>=cut, "prediction"] = "yes"
train_data.loc[train_data["rate"]<cut, "prediction"] = "no"

for i in range(train_data.shape[0]):
    if train_data.index[i] not in pats:
        if train_data["prediction"][i] == 'yes':
            train_data["prediction"][i] = 'no'

In [6]:
import numpy as np

prediction_0 = train_data
prediction_0.loc[prediction_0["prediction"]=="yes", "prediction"] = 1
prediction_0.loc[prediction_0["prediction"]=="no", "prediction"] = 0
prediction_0 = prediction_0.astype(np.int64)